# 1 Introduction

## Commentaires

### V2: Adaptation de la V1 sur train_V2 (meilleur modèle =best_LGBM)

- impact sur shap_values (LinearExplainer remplacé par Explainer (pb format array des shapvalues avec TreeExplainer))
- en // modification de la fonction def shap_value(cid:int): du repo P7_API_V2

### V1: Sur base train_V1

Cette version semble fonctionner: je réussis à appeler un SK_ID dans l'API qui me renvoie les shap.values

=> modèle splitté en 3: 

    - la partie sampler (qui elle ne sembe pas s'appliquer => j'ai toujours 61498 lignes dans mon df shap values)
    - la partie transformer (transfromer2) = imputer + standard scaler
    - la partie estimateur (LogReg2)

=> A inclure dans P7_2_1 pour vérifier quele shap.base est identique dans l'onglet 'Shap_test' et dans l'onglet 'shap_val

### Remarque: différence entre .loc et .iloc dans le résultat affiché dans l'API

Ave iloc pas d'index affiché / Avec .loc index affiché suivi de : puis donnée souhaitée (impact ajout d'une couche de dictionnaire)

=> Tester le passage du SK_ID dans l'index...

In [ ]:
pwd

## 1.1 Imports

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
#from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as Pipeline_imb
import lightgbm as lgb
from lightgbm import LGBMClassifier
import pickle
import shap
shap.initjs()

## 1.2 Fonctions utiles

In [ ]:
from src.fonctions import result, print_score, decomposition_modele

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
DISPLAY=True

"""If Dispaly == False then all cell starting starting with 'If Display=True' won't be ran.
Else they will

This short function allows to by-pass instructions which are long to run (ex MSNO on huge dataset or sns.pairplot)"""

# 2 Data

## 2.1 Import and display

In [ ]:
train_V2=pd.read_csv('../train_V2.csv')

In [ ]:
train_V2.shape

In [ ]:
train_V2=train_V2.sample(frac=0.2)
train_V2.shape

# 3 Best model: best_lgbm

In [ ]:
pkl_1= open("best_lgbm2.pkl","rb")
best_LGBM=pd.read_pickle(pkl_1)

In [ ]:
best_LGBM

In [ ]:
X=train_V2.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X.shape

In [ ]:
y=train_V2['TARGET']
y.shape

# 4 Adaptation modèle pour appel SHAP dans l'API

Rappel: Shap ne prend que l'estimateur (et pas le pipeline entier) pour calculer explaienr/shap_values

Or en récupérant le best_lgbm, et en le décomposant, dans l'API je dois faire appel à l'Id des crédits/clients. Cet Id n'est pas utilisé dans le best_lgbml.

2 solutions:

- 1/ j'ajoute l'id dans le bet_model, au risque d'avoir des résultats un peu moins bons => partie 4.1
- 2/ Comme pour l'AMNT_GOOD_PRICE (cf notebook ML), je reconstitue le df après l'entrainement => partie 4.2


## 4.1 Ajout de SK_ID_CURR après entrainement de best_model

- X: df sans SK_ID
- X2: df avec SK_ID

Etapes:

=> RandomUnderSampler sur X2
=> imputer sur X2
=> Suppression de SK_ID_CURR
=> transformer
=>fit LGBM

### New dataframe X2 avec SK_ID_CURR

In [ ]:
X2=train_V2.drop(['TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X2.shape


In [ ]:
y2=y
y2.shape

### Train test split sur base X2

In [ ]:
X2

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X2,y2,test_size=0.2)

In [ ]:
print('Train set:',X_train.shape)
print('Test set:',X_test.shape)
print('Train target:',y_train.shape)
print('Test target:',y_test.shape)

### Decomposition du best_lgm

In [ ]:
best_LGBM

In [ ]:
 #Sampling 
X_train_rus, y_train_rus=best_LGBM[0].fit_resample(X_train,y_train)
X_train_rus.shape

In [ ]:
# Fill Nan with simple imputer(median)
X_tr=pd.DataFrame(best_LGBM[1].fit_transform(X_train_rus),columns=X_train_rus.columns)
X_tr.shape

In [ ]:
# Suppression du SK_ID_CURR
X_tmp=X_tr.drop(['SK_ID_CURR'],axis=1)
X_tmp.shape

In [ ]:
best_LGBM[2]

In [ ]:
# Scaling
X_tr_transf=pd.DataFrame(best_LGBM[2].fit_transform(X_tmp), columns=X_tmp.columns)
X_tr_transf.shape

In [ ]:
X_tr_transf.isna().sum()

In [ ]:
# Entrainement
best_LGBM[-1].fit(X_tr_transf,y_train_rus)

Conclusion => décompostion du modèle en 3 étape:

-> Random Under Sampling + Imputer

-> Scaler

-> LGBM())

### Enregistrement pkl

In [ ]:
best_LGBM[:-2]

In [ ]:
best_LGBM[-2]

In [ ]:
pickle_out = open("samplerV2.pkl","wb")
pickle.dump(best_LGBM[:-2], pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open("transformer2_V2.pkl","wb")
pickle.dump(best_LGBM[-2], pickle_out)
pickle_out.close()

In [ ]:
pickle_out = open("est_LGBM.pkl","wb")
pickle.dump(best_LGBM[-1], pickle_out)
pickle_out.close()

# 5 Tests pour l'API:

Ci dessous: pas à pas de la fonction shap_value(cid:int) du repo P7_API_V2

In [ ]:
X2

In [ ]:
#X2=X2.drop(['SK_ID_CURR'],axis=1)
X2.shape

In [ ]:
transf_data = pd.DataFrame(best_LGBM[:-2].transform(X2),columns = X2.columns) # fit_resample necessite (X,y)
transf_data.shape

In [ ]:
data_for_shap=transf_data.drop(['SK_ID_CURR'],axis=1)

In [ ]:
data_for_shap_tr=pd.DataFrame(best_LGBM[-2].transform(data_for_shap),columns=data_for_shap.columns)

In [ ]:
best_LGBM[-1]

## Création de l'explaner Shap et des Shap_values

In [ ]:
explainer=shap.Explainer(best_LGBM[-1],data_for_shap_tr)

In [ ]:
type(explainer)

In [ ]:
shap_values = explainer(data_for_shap_tr,check_additivity=False)

In [ ]:
shap_values.cohorts

In [ ]:
type(shap_values)

In [ ]:
type(shap_values.values)

In [ ]:
type(shap_values.base_values)

In [ ]:
type(shap_values.data)

## Importance globale

In [ ]:
shap.summary_plot(shap_values,data_for_shap)

## Importance locale

In [ ]:
shap.plots.bar(shap_values,max_display=15)

In [ ]:
shap.plots.bar(shap_values[0],max_display=11)

In [ ]:
shap.waterfall_plot(shap_values[5], max_display=11)

In [ ]:
shap.waterfall_plot(shap_values[15], max_display=11)

In [ ]:

shap.force_plot(explainer.expected_value,
                explainer.shap_values(data_for_shap.iloc[5]),
                feature_names=data_for_shap.columns)
                #out_names="Conso")

In [ ]:
svv=pd.DataFrame(shap_values.values, columns = data_for_shap.columns).round(2)
svv

In [ ]:
svv['SK_ID_CURR']=transf_data['SK_ID_CURR']
svv